In [31]:
import h5py
import numpy as np
import os
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import netCDF4 as nc



In [63]:
month_date = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
prefix = "E://thesis_datasets//RADNL_CLIM_EM_MFBSNL25_05m_20161231T235500_20171231T235500_0002//RAD_NL25_RAC_MFBS_EM_5min//"

In [64]:
# Method 1
def eventSelection_maxN(N, t):
    # Ruben's way for event
    # Input: whole dataset; Output: picked sample's starting date
    # N: total number of sample we want
    # N/2 sample of highest average, N/2 sample of highest peak intensity
    # Loop through each possible starting time point
    # Step 1: calculate average precipitation
    # Step 2: update the ordered dict
    result = []
    result1 = {}
    result2 = {}
    year = "2017"
    for m in range(1, 13):
        for d in range(1, month_date[m] + 1):
            for h in range(24):
                for mi in range(0, 60, 30):  # every 30 mins
                    month = str(m) if m >= 10 else "0" + str(m)
                    date = str(d) if d >= 10 else "0" + str(d)
                    hour = str(h) if h >= 10 else "0" + str(h)
                    minute = str(mi) if mi >= 10 else "0" + str(mi)
                    start_time = year + month + date + hour + minute
                    print(start_time)
                    lead_times, obs_times = eventGeneration(start_time, obs_time=5, lead_time=t)
                    sample_times = obs_times + lead_times
                    if os.path.exists(prefix + year + "//" + month + "//" + \
                                   "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[0] + "_NL.h5") and \
                        os.path.exists(prefix + year + "//" + month + "//" + \
                                   "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[-1] + "_NL.h5"):

                        prep_average = 0
                        peak_average = 0
                        for time in sample_times:
                            filename = prefix + year + "//" + month + "//" + \
                                       "RAD_NL25_RAC_MFBS_EM_5min_" + time + "_NL.h5"
                            f = h5py.File(filename)['image1']['image_data']
                            f = np.array(f)
                            # f = np.where(f == 65535, -1, f)
                            # f = np.ma.masked_where(f <= 0, f)
                            f[f == 65535] = 0
                            f = f[100:500, 100:500] * 12 / 100
                            [rows, cols] = f.shape
                            prep_average += np.sum(f)/(rows*cols)
                            peak_average += np.max(f)
                        prep_average = prep_average/len(sample_times)
                        peak_average = peak_average/len(sample_times)

                        if len(result1)>=N:
                            result1_min = min(result1, key=result1.get)
                            if result1[result1_min] <= prep_average:
                                result1.pop(result1_min)
                                result1[start_time] = prep_average
                        else:
                            result1[start_time] = prep_average

                        if len(result2) >= N:
                            result2_min = min(result2, key=result2.get)
                            if result2[result2_min] <= peak_average:
                                result2.pop(result2_min)
                                result2[start_time] = peak_average
                        else:
                            result2[start_time] = peak_average

    result1 = sorted(result1.items(), key = lambda item:item[1], reverse=True)
    result2 = sorted(result2.items(), key = lambda item:item[1], reverse=True)
    i1 = 0
    i2 = 0
    while len(result) < N:
        while result1[i1][0] in result:
            i1+=1
        result.append(result1[i1][0])
        while result2[i2][0] in result:
            i2+=1
        result.append(result2[i2][0])
    with open("selectedTime.txt", "w") as f:
        for r in result:
            f.write(r + "\n")
    return result


In [67]:
# Method 2
def eventSelection_threshold(X, Y, Z, A, B, t):
    # E.C. thesis's event selection
    # Input: whole dataset; Output: picked sample's starting date
    # Loop through each frame and label the frame X% >= A, Y%>=B
    # Loop through each possible starting time point
    # Label the sequence, >Z heavy rainfall
    result_light = []
    result_heavy = []
    year = "2008"
    for m in range(7, 8):
        for d in range(3, 11):
            for h in range(24):
                for mi in range(0, 60, 30):  # every 30 mins
                    month = str(m) if m >= 10 else "0" + str(m)
                    date = str(d) if d >= 10 else "0" + str(d)
                    hour = str(h) if h >= 10 else "0" + str(h)
                    minute = str(mi) if mi >= 10 else "0" + str(mi)
                    start_time = year + month + date + hour + minute
                    #print(start_time)
                    lead_times, obs_times = eventGeneration(start_time, obs_time=5, lead_time=t)
                    sample_times = obs_times + lead_times
                    sample_label = { i : 0 for i in sample_times }
                    if os.path.exists(prefix + year + "//" + month + "//" + \
                                      "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[0] + "_NL.h5") and \
                            os.path.exists(prefix + year + "//" + month + "//" + \
                                           "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[-1] + "_NL.h5"):
                        heavy_frame = 0
                        for time in sample_times:
                            filename = prefix + year + "//" + month + "//" + \
                                           "RAD_NL25_RAC_MFBS_EM_5min_" + time + "_NL.h5"
                            f = h5py.File(filename)['image1']['image_data']
                            f = np.array(f)
                            f = np.where(f == 65535, -1, f)
                            f = f[100:500, 100:500] * 12 / 100
                            f_light = np.ma.masked_where(f <= A, f)
                            f_heavy = np.ma.masked_where(f <= B, f)
                            [rows, cols] = f.shape
                            rc = rows*cols
                            nLight = f_light.count()
                            nHeavy = f_heavy.count()
                            if nLight/rc >= X/100:
                                sample_label[time] = 1
                                if nHeavy/rc >= Y/100:
                                    sample_label[time] = 2
                                    heavy_frame += 1
                            else:
                                break
                        if heavy_frame >= Z:
                            result_heavy.append(start_time)
                        else:
                            result_light.append(start_time)
    return result_light, result_heavy

In [71]:
# Method 3
def eventSelection_importance(p_s, p_m, t, q_min):
    # Importance sampling scheme, Nature
    # Input: whole dataset; Output: picked sample's starting date
    # Loop through each possible starting time point
    # Step 1: generate t*h*w sample
    # Step 2: calculate the probability of sample: q_n = min{1, q_min + (m/c)*sum(x_sat)}, x_sat = 1-exp(-x/s)
    # Step 3: Store the data
    result = []
    year = "2008"
    for m in range(7, 8):
        for d in range(3, 32):
            for h in range(24):
                for mi in range(0, 60, 30):  # every 30 mins
                    month = str(m) if m >= 10 else "0" + str(m)
                    date = str(d) if d >= 10 else "0" + str(d)
                    hour = str(h) if h >= 10 else "0" + str(h)
                    minute = str(mi) if mi >= 10 else "0" + str(mi)
                    start_time = year + month + date + hour + minute
                    lead_times, obs_times = eventGeneration(start_time, obs_time=5, lead_time=t)
                    sample_times = obs_times + lead_times
                    x_sat_sum = 0
                    if os.path.exists(prefix + year + "//" + month + "//" + \
                                   "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[0] + "_NL.h5") and \
                        os.path.exists(prefix + year + "//" + month + "//" + \
                                   "RAD_NL25_RAC_MFBS_EM_5min_" + sample_times[-1] + "_NL.h5"):

                        for time in sample_times:
                            filename = prefix + year + "//" + month + "//" + \
                                       "RAD_NL25_RAC_MFBS_EM_5min_" + time + "_NL.h5"
                            f = h5py.File(filename)['image1']['image_data']
                            f = np.array(f)
                            # f = np.where(f == 65535, -1, f)
                            # f = np.ma.masked_where(f <= 0, f)
                            f[f == 65535] = 0
                            f = f[100:500, 100:500] * 12 / 100
                            [rows, cols] = f.shape
                            x_sat = np.sum(1-np.exp((-1/p_s)*f))
                            x_sat_sum += x_sat
                        q = min(1, q_min + (p_m*x_sat_sum/(rows*cols*(t+5))))
                        #print(start_time, " probability:", q)
                        picked = random.choices([True,False], weights=[q,1-q])
                        if picked: result.append(start_time)
    return result


In [34]:
# utils
def eventGeneration(start_time, obs_time = 4 ,lead_time = 72):
    # Generate event based on starting time point, return a list: [[t-4,...,t-1,t], [t+1,...,t+72]]
    # Get the start year, month, day, hour, minute
    year = int(start_time[0:4])
    month = int(start_time[4:6])
    day = int(start_time[6:8])
    hour = int(start_time[8:10])
    minute = int(start_time[10:12])
    #print(datetime(year=year, month=month, day=day, hour=hour, minute=minute))
    times = [(datetime(year, month, day, hour, minute) + timedelta(minutes=5 * (x+1))) for x in range(lead_time)]
    lead = [dt.strftime('%Y%m%d%H%M') for dt in times]
    times = [(datetime(year, month, day, hour, minute) - timedelta(minutes=5 * x)) for x in range(obs_time)]
    obs = [dt.strftime('%Y%m%d%H%M') for dt in times]
    obs.reverse()
    return lead, obs


In [65]:
# Function test
#lead, obs = eventGeneration('200910080600')
#print(lead)
'''
result = eventSelection_importance(1, 0.1, 10, 2e-4)
print(result)

result_light, result_heavy = eventSelection_threshold(X = 20, Y = 5, Z = 10, A = 1, B = 20, t=20)
print(result_heavy)
print(len(result_heavy))
print(result_light)
print(len(result_light))
'''
result_max = eventSelection_maxN(100, 12)

print(result_max)


201701010000
201701010030


C:\Users\zywang\AppData\Local\Temp/ipykernel_289032/631076515.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(filename)['image1']['image_data']


201701010100
201701010130
201701010200
201701010230
201701010300
201701010330
201701010400
201701010430
201701010500
201701010530
201701010600
201701010630
201701010700
201701010730
201701010800
201701010830
201701010900
201701010930
201701011000
201701011030
201701011100
201701011130
201701011200
201701011230
201701011300
201701011330
201701011400
201701011430
201701011500
201701011530
201701011600
201701011630
201701011700
201701011730
201701011800
201701011830
201701011900
201701011930
201701012000
201701012030
201701012100
201701012130
201701012200
201701012230
201701012300
201701012330
201701020000
201701020030
201701020100
201701020130
201701020200
201701020230
201701020300
201701020330
201701020400
201701020430
201701020500
201701020530
201701020600
201701020630
201701020700
201701020730
201701020800
201701020830
201701020900
201701020930
201701021000
201701021030
201701021100
201701021130
201701021200
201701021230
201701021300
201701021330
201701021400
201701021430
201701021500

In [21]:
event_2008 = ['200808032230', '200806221300', '200808032200', '200807261830', '200811102300', '200807261700', '200811102330', '200806051730', '200808032300', '200807261600', '200811110000', '200807261530', '200803202000', '200807261630', '200803202030', '200807261800', '200803202100', '200807261900', '200811102230', '200807261730', '200808071300', '200806221230', '200803201930', '200806221330', '200811110030', '200807262030', '200808071230', '200807261500', '200808032130', '200806022030', '200803202130', '200806221200', '200808071330', '200807262100', '200811110100', '200807261930', '200808032330', '200806022000', '200801211630', '200806022100', '200810051230', '200807262000', '200810051300', '200807291300', '200803201900', '200806051700', '200811110130', '200808071400', '200811102200', '200807291230', '200811110200', '200806221130', '200810051330', '200807290330', '200811110230', '200807261430', '200803010230', '200807270030', '200801211700', '200807020400', '200803202200', '200806022130', '200801211600', '200809031400', '200810051600', '200807020330', '200810051200', '200807291330', '200810051530', '200807270100', '200810051400', '200806060330', '200811110300', '200807280130', '200810051500', '200806022200', '200810051430', '200809120830', '200803010200', '200809031330', '200811110500', '200808070830', '200808032100', '200807020100', '200810051630', '200809031430', '200811110400', '200809031500', '200811110330', '200806071430', '200811110430', '200808070800', '200803201830', '200806060300', '200810051130', '200806022230', '200808071200', '200808071100', '200809121130', '200807060100']
print(len(event_2008))

100


In [60]:
event_2009 = ['200905260230', '200905260130', '200905260200', '200907031400', '200905260300', '200907031330', '200905260330', '200905260100', '200911011530', '200907211530', '200910071730', '200907211900', '200911011600', '200907031430', '200911011500', '200905260030', '200910071700', '200907031300', '200907222200', '200905251930', '200910071800', '200905162030', '200911011630', '200910111700', '200907222230', '200907061730', '200911231700', '200907211830', '200907222130', '200908201630', '200911231730', '200907211600', '200905260400', '200905162000', '200907222300', '200907061800', '200910071630', '200907142200', '200911011430', '200910111630', '200911011700', '200907031230', '200910080100', '200907142230', '200907222330', '200907211500', '200910080130', '200907051800', '200910071830', '200908201600', '200911231630', '200905260000', '200907222100', '200907061700', '200909042000', '200905251900', '200909041930', '200901030830', '200911011730', '200907031500', '200910080030', '200901030900', '200905151700', '200908201700', '200907100830', '200907051700', '200905151630', '200907051730', '200909042030', '200907190730', '200907100800', '200907051830', '200907100900', '200908201730', '200907230000', '200910111730', '200902092300', '200905252000', '200905151730', '200906091430', '200910080200', '200907211930', '200909041900', '200907061830', '200910071600', '200901030800', '200905151600', '200907292300', '200905151800', '200907190700', '200911231800', '200907170130', '200907100930', '200908012200', '200912250230', '200907140200', '200907100730', '200907051600', '200902092330', '200908201530']
print(len(event_2009))

100


In [54]:
event_2011 = ['201109062100', '201106281800', '201109062130', '201106281830', '201109062030', '201106281730', '201109062200', '201109102130', '201107122000', '201109102100', '201107122030', '201109102200', '201109062000', '201108231030', '201107121930', '201106061430', '201107122100', '201108231100', '201106282000', '201108230930', '201107122130', '201106061400', '201107121900', '201108061330', '201109062230', '201108231000', '201106281930', '201106281900', '201109061930', '201108230900', '201107122200', '201109102030', '201106282030', '201108061400', '201107121830', '201106061500', '201112082200', '201108061300', '201107122230', '201107281630', '201110180600', '201107281700', '201106160700', '201109102230', '201110180630', '201108251630', '201106282100', '201107281530', '201106282130', '201108260600', '201108140430', '201108261100', '201106282200', '201108261030', '201106160730', '201108260630', '201108140500', '201107281600', '201112082230', '201108181600', '201112082130', '201108230830', '201106160630', '201108061430', '201101130800', '201108261000', '201109061900', '201107281500', '201101130830', '201108061230', '201106160800', '201106111400', '201108140530', '201105301430', '201108140400', '201109102000', '201106160600', '201109032230', '201106282230', '201109032300', '201112130300', '201108261130', '201110180700', '201109102300', '201112130330', '201106061530', '201109062300', '201108260930', '201110180530', '201108251400', '201107122300', '201106061600', '201107121800', '201108251700', '201112140630', '201106051400', '201108140600', '201104241830', '201112140600', '201108251600']
print(len(event_2011))

100


In [36]:
event_2013 = ['201309080600', '201308052130', '201309080630', '201307270900', '201309080530', '201308052200', '201309080700', '201307270830', '201307272300', '201308052100', '201307270930', '201308052230', '201309101730', '201308052030', '201309080500', '201306201300', '201307272330', '201307272230', '201309101700', '201307021900', '201307271000', '201306201230', '201309101800', '201307021930', '201307271100', '201306201330', '201307280000', '201307272200', '201307271030', '201308191100', '201309080730', '201307280030', '201309101630', '201307272130', '201309101830', '201308052000', '201311040300', '201308190630', '201301300700', '201308190600', '201311040330', '201308191130', '201307271130', '201308190700', '201310280300', '201307260830', '201306210530', '201308190530', '201311091800', '201307260900', '201310280330', '201308190900', '201311040230', '201307270800', '201311091830', '201305291330', '201309101600', '201310040530', '201309101900', '201311031330', '201306210600', '201307231700', '201311040400', '201308052300', '201310280230', '201308111800', '201305161530', '201307290200', '201309080430', '201308191030', '201306210500', '201306201200', '201305161500', '201305291400', '201306201400', '201307231630', '201311091730', '201308111330', '201311091900', '201308190500', '201309080800', '201307290230', '201310130730', '201307261200', '201310130800', '201308111730', '201307271200', '201308190730', '201301300730', '201307260800', '201309152130', '201308190430', '201310130700', '201307231730', '201311040200', '201307260930', '201311040430', '201311031400', '201301300630', '201307251830']
print(len(event_2013))

100


In [48]:
event_2015 = ['201508261900', '201508302230', '201508261930', '201508302200', '201508261830', '201508302300', '201501081230', '201508241530', '201501081300', '201508301900', '201508262000', '201508301930', '201501081200', '201507050900', '201501081330', '201507050930', '201501081130', '201507021930', '201508261800', '201508241500', '201501081400', '201507051330', '201503291430', '201508241600', '201503302330', '201508131900', '201508302130', '201507022000', '201503291500', '201508140030', '201503291400', '201508241430', '201508262030', '201507051400', '201511150230', '201508241630', '201511150300', '201506051800', '201507251430', '201506051730', '201505051130', '201508302330', '201507251500', '201508131830', '201507251400', '201507051000', '201503302300', '201507051430', '201505051100', '201508140130', '201503291530', '201508261730', '201508171830', '201508300300', '201501281400', '201508140000', '201501281430', '201507021900', '201501081100', '201505051200', '201511150200', '201508140100', '201508171900', '201505051030', '201501281330', '201507051300', '201501081430', '201508302000', '201507251330', '201508150030', '201503291330', '201508150000', '201511150330', '201508131930', '201511150130', '201508241400', '201508171930', '201507022030', '201508171800', '201506051700', '201507270900', '201508241330', '201511291900', '201507050830', '201501091630', '201508132330', '201507270930', '201508241700', '201507131030', '201508150100', '201511291830', '201507051500', '201501281500', '201508151330', '201501091700', '201508300230', '201507131100', '201508300200', '201501281300', '201506051830']
print(len(event_2015))

100


In [66]:
event_2017 = ['201706090830', '201709291600', '201706090800', '201709291630', '201706090900', '201709111730', '201706090730', '201709111700', '201707120700', '201707200630', '201707120630', '201707280830', '201706090700', '201709291530', '201707120600', '201708301700', '201707120530', '201708301730', '201707120500', '201709111630', '201707120900', '201709291700', '201707120730', '201706030400', '201706090930', '201707201500', '201707120930', '201707141000', '201701122330', '201707200700', '201707120830', '201707280800', '201701130000', '201707141030', '201707120800', '201709111800', '201707120430', '201707201430', '201706090630', '201707201230', '201701130030', '201707201300', '201701122300', '201706030330', '201701122030', '201707140930', '201706281400', '201709291730', '201707292030', '201708301800', '201707292000', '201708181730', '201706281430', '201706150300', '201710071730', '201707071500', '201712071530', '201707292130', '201712071500', '201708181800', '201710071800', '201707141100', '201710071700', '201707201200', '201701122000', '201706030430', '201701122230', '201707292200', '201706281330', '201707201400', '201706281500', '201705140400', '201710071830', '201707170430', '201707291930', '201709291800', '201701122100', '201707191030', '201703172230', '201708301630', '201707291830', '201707222330', '201707291900', '201707280730', '201707121000', '201708181830', '201709130100', '201707201330', '201703172300', '201709112330', '201707120400', '201706150230', '201707292100', '201708181900', '201709130130', '201707201130', '201712071600', '201707200600', '201701122200', '201707062300']
print(len(event_2017))

100


In [22]:

string = event_2008[0]
print(string[0:3])
num = int(string[4:6])
print(num)

200
8


In [107]:
def string_to_number(string):
    year = int(string[0:4])
    month = int(string[4:6])
    date = int(string[6:8])
    hour = int(string[8:10])
    minute = int(string[10:12])
    return[year, month, date, hour, minute]
    

In [41]:
def search_carrots_index(month, date):
    index = 0
    month_date = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    for i in range(month):
        index = index + month_date[i]
    index = index + date
    index = index + 1
    
    return index

In [42]:
def calculate_mse_and_mae(f_origin, f_processed):
    MSE = 0
    MSE1 = 0
    MAE = 0
    MAE1 = 0

    '''
    year = 2013
    month = 6
    date = 20
    hour = 13
    minute = 0
    '''

    i = 0

    for x in range(0, 765):
        for y in range(0, 700):
            if f_origin[x][y] < 5000 and f_processed[x][y] < 5000:
                MSE = MSE + (f_processed[x][y] - f_origin[x][y])**2
                MAE = MAE + abs(f_processed[x][y] - f_origin[x][y])

                i = i + 1


    if i != 0:
        MSE = MSE / i
        MAE = MAE / i
    
    return [MSE, MAE]

In [27]:
#carrots factor
factor = np.zeros((368, 765, 700))
file_name_CARROTS = "E:/thesis_code/CARROT/RadarBiasCorrectionFactors_NL.nc"
factor1 = nc.Dataset(file_name_CARROTS)
factor1 = factor1['factor']

for i in range(368):
    factor[i] = 1 / factor1[i]

In [67]:
# Calculate MSE and MAE for extreme events
MSE = 0
MSE_carrot = 0
MAE = 0
MAE_carrot = 0
i = 1
month_date = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    


for event in event_2017:
    year = int(event[0:4])
    month = int(event[4:6])
    date = int(event[6:8])
    hour = int(event[8:10])
    minute = int(event[10:12])
    
    index = str(i)        
    print("Event" + index)
    i = i + 1
    
 
    index = search_carrots_index(month, date)

    if month < 10:
        mth = "0" + str(month)
    else:
        mth = str(month)

    if date < 10:
        dt = "0" + str(date)
    else:
        dt = str(date)
                
    if hour < 10:
        hr = "0" + str(hour)
    else:
        hr = str(hour)
    if minute < 10:
        mnt = "0" + str(minute)
    else:
        mnt = str(minute)

    print(str(year) + "." + mth + "." + dt + " " + hr + ":" + mnt )

    filename_MFBS = "E://thesis_datasets/RADNL_CLIM_EM_MFBSNL25_05m_" + str(year - 1) + "1231T235500_" + str(year) + "1231T235500_0002/RAD_NL25_RAC_MFBS_EM_5min/" + str(year) + "/" + mth + "/" + \
                    "RAD_NL25_RAC_MFBS_EM_5min_" + str(year) + mth + dt + hr + mnt + "_NL.h5"
    #print(filename_MFBS)

    f = h5py.File(filename_MFBS)['image1']['image_data']
    f = np.array(list(f))
    #f = np.where(f == 65535, -1, f) 
    #f = np.ma.masked_where(f <= 0, f)
    f = f / 100
    f = f * 12
    filename_RT = "F://"+ str(year) + "//" + mth + "//" + \
    "RAD_NL25_RAP_5min_" + str(year) + mth + dt + hr + mnt + ".h5"
    f1 = h5py.File(filename_RT)['image1']['image_data']
    f1 = np.array(list(f1))
    #f1 = np.where(f1 == 65535, -1, f1) 
    #f1 = np.ma.masked_where(f1 <= 0, f1)
    f1 = f1 / 100
    f1 = f1 * 12

    [MSE_temp, MAE_temp] = calculate_mse_and_mae(f, f1)
    MSE = MSE + MSE_temp
    MAE = MAE + MAE_temp
    print("Current MSE is:" + str(MSE))
    print("Current MAE is:" + str(MAE))

    processed_image = np.zeros((765, 700))
    for x in range(0, 765):
        for y in range(0, 700):
            if(f1[x][y] < 5000):
                processed_image[x][y] = f1[x][y] * factor[index][764- x][y]

    [MSE_temp, MAE_temp] = calculate_mse_and_mae(f, processed_image)
    MSE_carrot = MSE_carrot + MSE_temp
    MAE_carrot = MAE_carrot + MAE_temp
    print("Current MSE_carrot is:" + str(MSE_carrot))
    print("Current MAE_carrot is:" + str(MAE_carrot))

Event1
2017.06.09 08:30


C:\Users\zywang\AppData\Local\Temp/ipykernel_289032/3362828198.py:50: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(filename_MFBS)['image1']['image_data']
C:\Users\zywang\AppData\Local\Temp/ipykernel_289032/3362828198.py:58: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f1 = h5py.File(filename_RT)['image1']['image_data']


Current MSE is:5.952125126032252
Current MAE is:1.612186081558788
Current MSE_carrot is:2.1948550657383636
Current MAE_carrot is:0.9138284872143534
Event2
2017.09.29 16:00
Current MSE is:13.619850374062832
Current MAE is:2.0145829670429283
Current MSE_carrot is:5.735992536163881
Current MAE_carrot is:1.1390400099240512
Event3
2017.06.09 08:00
Current MSE is:17.52891339995003
Current MAE is:3.3175083873052453
Current MSE_carrot is:6.838128235458945
Current MAE_carrot is:1.7863806253045902
Event4
2017.09.29 16:30
Current MSE is:85.40332851324239
Current MAE is:4.695877572256554
Current MSE_carrot is:55.44219404366092
Current MAE_carrot is:2.9035647985926927
Event5
2017.06.09 09:00
Current MSE is:91.8090904537132
Current MAE is:6.312845225067113
Current MSE_carrot is:57.80305124466505
Current MAE_carrot is:3.8169737453573807
Event6
2017.09.11 17:30
Current MSE is:100.61619106190446
Current MAE is:7.035460987818512
Current MSE_carrot is:59.3012087375619
Current MAE_carrot is:4.094475418095

In [68]:
print("100 extreme events in 2017")
MSE = MSE/100
MAE = MAE/100
MSE_carrot = MSE_carrot/100
MAE_carrot = MAE_carrot/100

print("MSE is:" + str(MSE))
print("MAE is:" + str(MAE))
print("MSE_carrot is:" + str(MSE_carrot))
print("MAE_carrot is:" + str(MAE_carrot))

100 extreme events in 2017
MSE is:7.683546404204564
MAE is:0.9377686431403813
MSE_carrot is:3.5825538518944935
MAE_carrot is:0.5521421871296446
